In [94]:
import pandas as pd
from pandas.tseries.offsets import BDay
import yfinance as yf
import numpy as np
import math
from tqdm import tqdm

In [95]:
def backtest_basket(strikes, tickers_basket):
    #Here we will consider a business month expiration strategy (21 day in a business day convention).
    df_backtest = yf.download(tickers = tickers_basket)["Close"].pct_change(periods  = 21).apply(lambda x: x + 1)
    df_backtest.dropna(inplace = True)
    counter = 0
    #We are going to check if every condition 
    while counter < len(strikes):
        #Initialization
        if counter == 0:
            #Down Condition
            if strikes[counter] <= 100:
                mask = df_backtest.iloc[:, counter]<=strikes[counter]/100
            else: 
                mask = df_backtest.iloc[:, counter]>=strikes[counter]/100
        else:
            if strikes[counter] <= 100:
                mask = np.logical_and(mask, df_backtest.iloc[:, counter]<=strikes[counter]/100)
            else: 
                mask = np.logical_and(mask, df_backtest.iloc[:, counter]>=strikes[counter]/100)
        counter += 1
    df_backtest["Profit"] = mask
    profits_proportion = sum(df_backtest["Profit"]/len(df_backtest))
    return df_backtest, profits_proportion

In [96]:
df = backtest_basket([102, 99], ["AFL", "ZION"])[0]
c = df.cov()*math.sqrt(252)
c.loc["ZION", "ZION"]

[*********************100%***********************]  2 of 2 completed


0.15992319886606568

In [119]:
def simulate_paths(tickers, NTS, T, N=100):
    """
    Simulating realizations of log-normal risk-neutral random walk

    :param S0: Initial asset price
    :param drift_rate: annualized return of the asset
    :param volatility: annualized daily vol
    :param NTS: number of time steps
    :param T: Derivatives expiration (in years)
    :param N: Number of simulated paths for each ticker

    :return:  
    """
    df = yf.download(tickers = tickers, start = "2020-06-01")["Close"]
    drift = df.pct_change().mean()*252
    cov_matrix = df.pct_change().cov()*math.sqrt(252)
    realization_dict = {ticker : np.zeros((NTS, N)) for ticker in tickers}
    dt = T / NTS
    # We will start at S0 for every simulation
    for ticker in tickers:
        S0 = df[ticker][-1]
        realization_dict[ticker][0, :] = [S0 for simulation in range(N)] 
    #Now that everything is set-up let us simulate (we will simulate a realization of a multivariable normal
    # random variable with mean  = drift and variances = 1 covariated as the returns of each asset. 
    for simulation in tqdm(range(N)):
        for timestep in range(1, NTS):
            random_variable = np.random.multivariate_normal(np.zeros(len(tickers)), cov_matrix)
            for ticker in range(len(tickers)):
                realization_dict[tickers[ticker]][timestep, simulation] = realization_dict[tickers[ticker]][timestep - 1, simulation] \
                                                          * math.exp((drift[ticker] - 1 / 2 * cov_matrix.iloc[ticker, ticker] ** 2) * dt
                                                                     + math.sqrt(dt) * random_variable[
                                                                         ticker])
    return realization_dict

In [132]:
number_paths = 500
tickers = ["AFL", "ZION"]
simulate = simulate_paths(tickers = tickers, NTS = 1000, T = 1/12, N = number_paths)
strikes = [98, 102]
#We will check for every path if the condition is met. 
counter = 0
for path in range(number_paths):
    print("Veryfing path number", path)
    for i in range(len(tickers)):
        ticker = tickers[i]
        strike = strikes[i]
        if strikes[i] <=100:
            #On every path we will check the returns made by the asset on one month.
            if (simulate[ticker][-1, path] - simulate[ticker][0, path])/simulate[ticker][0, path] >= (100 - strike)/100:
                print("DOWN", (simulate[ticker][-1, path] - simulate[ticker][0, path])/simulate[ticker][0, path])
                break
        else:
            if (simulate[ticker][-1, path] - simulate[ticker][0, path])/simulate[ticker][0, path] <= (100 - strike)/100:
                print("UP", (simulate[ticker][-1, path] - simulate[ticker][0, path])/simulate[ticker][0, path])
                break
        print("This path works well! ", path)
        counter += 1

[*********************100%***********************]  2 of 2 completed


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:22<00:00, 22.71it/s]

Veryfing path number 0
This path works well!  0
This path works well!  0
Veryfing path number 1
This path works well!  1
This path works well!  1
Veryfing path number 2
DOWN 0.02436032299369529
Veryfing path number 3
This path works well!  3
UP -0.039269113531645486
Veryfing path number 4
This path works well!  4
This path works well!  4
Veryfing path number 5
DOWN 0.03748082498240718
Veryfing path number 6
This path works well!  6
UP -0.02563701901663057
Veryfing path number 7
DOWN 0.050846297974899105
Veryfing path number 8
DOWN 0.022314324338395662
Veryfing path number 9
This path works well!  9
This path works well!  9
Veryfing path number 10
DOWN 0.022920394010224724
Veryfing path number 11
DOWN 0.026464251165912143
Veryfing path number 12
DOWN 0.037623110130600594
Veryfing path number 13
DOWN 0.0446424101821609
Veryfing path number 14
This path works well!  14
This path works well!  14
Veryfing path number 15
DOWN 0.02618547476556344
Veryfing path number 16
This path works well! 

In [133]:
counter

401